In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv2D, MaxPooling2D, Flatten, MaxPooling1D, Conv1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

import tensorflow_addons as tfa
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df=pd.read_csv('input9.csv')

In [3]:
df.head()

,comment,class
0,FIXME formatters are not thread safe,DEFECT
1,"XXX Jon Skeet The comment ""if it hasn't bee...",DEFECT
2,"I hate to admit it, but we don't know what h...",DEFECT
3,Just a note StarTeam has a status for NEW wh...,DEFECT
4,the generated classes must not be added in t...,DEFECT


In [4]:
df['class'].value_counts()

WITHOUT_CLASSIFICATION    58204
DESIGN                     2703
IMPLEMENTATION              757
DEFECT                      472
TEST                         85
DOCUMENTATION                54
Name: class, dtype: int64

In [5]:
df['class']=df['class'].apply(lambda x: 0 if x=='WITHOUT_CLASSIFICATION' else 1)

In [6]:
df['class'].value_counts()

0    58204
1     4071
Name: class, dtype: int64

In [8]:
counts = df['class'].value_counts()

weight_dict = {0: 1.0, 1: counts[0] / counts[1]}

print(weight_dict)

{0: 1.0, 1: 14.29722426922132}


In [9]:
X = df['comment']
Y = df['class']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, stratify=Y, random_state=42)

max_words = 10000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
train_sequences = tok.texts_to_sequences(X_train)
train_sequences_matrix = pad_sequences(train_sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [11]:
print(len(train_sequences_matrix[0]))

250


In [12]:
def CNN():
    cnn = tf.keras.Sequential()
    cnn.add(Input(name='inputs',shape=[max_len]))
    cnn.add(Embedding(max_words,50,input_length=max_len))
    cnn.add(Conv1D(128, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(256, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(512, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))

    cnn.add(Flatten())
    cnn.add(Dense(256, activation ='relu'))
    cnn.add(Dense(64, activation ='relu'))
    cnn.add(Dense(1, activation='sigmoid'))

    return cnn

In [13]:
model = CNN()
model.summary()
opt = tf.keras.optimizers.Adadelta(learning_rate=0.1)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 50)           500000    
                                                                 
 conv1d (Conv1D)             (None, 250, 128)          19328     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 125, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 125, 256)          98560     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 62, 256)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 62, 512)           3

In [14]:
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_loss',
                mode='min',
                save_best_only=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

model.fit(train_sequences_matrix,Y_train,batch_size=128,epochs=50,validation_split=0.25, callbacks=[model_checkpoint_callback,reduce_lr], class_weight=weight_dict)

model.load_weights(checkpoint_filepath)

Epoch 1/50
292/292 [==============================] - 12s 25ms/step - loss: 0.9948 - accuracy: 0.7266 - val_loss: 0.1667 - val_accuracy: 0.9656 - lr: 0.1000
Epoch 2/50
292/292 [==============================] - 6s 22ms/step - loss: 0.4874 - accuracy: 0.9262 - val_loss: 0.2751 - val_accuracy: 0.9088 - lr: 0.1000
Epoch 3/50
292/292 [==============================] - 6s 22ms/step - loss: 0.3786 - accuracy: 0.9352 - val_loss: 0.1911 - val_accuracy: 0.9358 - lr: 0.1000
Epoch 4/50
292/292 [==============================] - 6s 22ms/step - loss: 0.3100 - accuracy: 0.9462 - val_loss: 0.3086 - val_accuracy: 0.8792 - lr: 0.1000
Epoch 5/50
292/292 [==============================] - 7s 23ms/step - loss: 0.2677 - accuracy: 0.9535 - val_loss: 0.1256 - val_accuracy: 0.9589 - lr: 0.1000
Epoch 6/50
292/292 [==============================] - 6s 22ms/step - loss: 0.2308 - accuracy: 0.9592 - val_loss: 0.1404 - val_accuracy: 0.9531 - lr: 0.1000
Epoch 7/50
292/292 [==============================] - 7s 23ms/s

In [15]:
model.save("CNN_imbalanced_weighted.h5")

In [16]:
accr = model.evaluate(test_sequences_matrix,Y_test)

390/390 [==============================] - 2s 6ms/step - loss: 0.1052 - accuracy: 0.9694


In [17]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.105
  Accuracy: 0.969


In [18]:
y_pred = model.predict(test_sequences_matrix)

390/390 [==============================] - 2s 5ms/step


In [19]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [20]:
print(classification_report(Y_test, y_pred, digits=6))

              precision    recall  f1-score   support

           0   0.991531  0.975603  0.983503     11641
           1   0.716284  0.880835  0.790083       814

    accuracy                       0.969410     12455
   macro avg   0.853908  0.928219  0.886793     12455
weighted avg   0.973542  0.969410  0.970862     12455

